In [2]:
import numpy as np 

import cv2 as cv 

 

# Defining Normal Probability Density Function with parameters x, mean and standard deviation. 

 

def norm_pdf(x,mean,sigma): 

    return (1/(np.sqrt(2*3.14)*sigma))*(np.exp(-0.5* (((x-mean)/sigma)**2))) 

 

# Receiving the Video and converting the coloured video (with RGB) into Gray scale. 

capture = cv.VideoCapture("umcp.mpg")                                   

ret,color_frame = capture.read() 

frame = cv.cvtColor(color_frame,cv.COLOR_BGR2GRAY) 

 

 

# Defining the matrix with rows and columns 

row,col = frame.shape 

# Initialising the mean, variance, omega and omega by sigma 

mean = np.zeros([3,row,col],np.float64) 

mean[1,:,:] = frame 

 

variance = np.zeros([3,row,col],np.float64) 

variance[:,:,:] = 225 

# Omega is an estimate of the weight (the portion of the data accounted for by this Gaussian). 

omega = np.zeros([3,row,col],np.float64) 

omega[:,:,:],omega[:,:,:],omega[:,:,:] = 0.3,0.3,0.4 

 

omega_by_sigma = np.zeros([3,row,col],np.float64) 

# Initialising Foreground as fg and Background as bg 

fg = np.zeros([3,row,col],np.uint8) 

bg = np.zeros([3,row,col],np.uint8) 

bg = frame 

 

#Initialising T which is ‘percentage of data that should be accounted for background’ and alpha is ‘learning rate’ which are chosen arbitrarily. 

alpha = 0.8

T = 0.3 

 

fourcc = cv.VideoWriter_fourcc(*'MJPG') 

#cv.VideoWriter(‘Filename’,  Source ,fps, framesize, color) 

out_1= cv.VideoWriter('bg.avi',fourcc, 20, (352,240),isColor=False) 

out_2 = cv.VideoWriter('fg.avi',fourcc, 20, (352,240),isColor=False) 

# Initialising the while loop. 

while capture.isOpened(): 

    ret,frame = capture.read() 

     

    if cv.waitKey(1) & 0xFF == 27 or ret == False: 

        break 

    Gray_Frame = cv.cvtColor(frame,cv.COLOR_BGR2GRAY) 

     

    Gray_Frame = Gray_Frame.astype(np.float64) 

    # The variances of those indices become negative as a result of using the norm_pdf function, so we convert them based on their preferences, they may have values between zero and some higher values. 

    variance[0][np.where(variance[0]<1)] = 1 

    variance[1][np.where(variance[1]<1)] = 1 

    variance[2][np.where(variance[2]<1)] = 1 

     

 

    # Calulating standard deviation 

    sigma1 = np.sqrt(variance[0]) 

    sigma2 = np.sqrt(variance[1]) 

    sigma3 = np.sqrt(variance[2]) 

 

    # Getting values for fitting indexes using inequality test. 

    comp_value_1 = cv.absdiff(Gray_Frame,mean[0]) 

    comp_value_2 = cv.absdiff(Gray_Frame,mean[1]) 

    comp_value_3 = cv.absdiff(Gray_Frame,mean[2]) 

 

# A match is defined as a pixel value within 2.5 standard deviations of a distribution 

    value_1 = 2.5 * sigma1 

    value_2 = 2.5 * sigma2 

    value_3 = 2.5 * sigma3 

     

    # Identifying those indices where a particular pixel values fits at least one of the gaussian 

    gaussian_fit_index1 = np.where(comp_value_1 <= value_1) 

    gaussian_not_fit_index1 = np.where(comp_value_1 > value_1) 

 

    gaussian_fit_index2 = np.where(comp_value_2 <= value_2) 

    gaussian_not_fit_index2 = np.where(comp_value_2 > value_2) 

 

    gaussian_fit_index3 = np.where(comp_value_3 <= value_3) 

    gaussian_not_fit_index3 = np.where(comp_value_3 > value_3) 

     

     

    match_index = np.zeros([row,col]) 

    match_index[gaussian_fit_index1] = 1 

    match_index[gaussian_fit_index2] = 1 

    match_index[gaussian_fit_index3] = 1 

    not_match_index = np.where(match_index == 0) 

     

     

    # Updating variance and mean value of the matched indices of all three gaussians 

    rho = alpha * norm_pdf(Gray_Frame[gaussian_fit_index1], mean[0][gaussian_fit_index1], sigma1[gaussian_fit_index1]) 

    C = rho * ((Gray_Frame[gaussian_fit_index1] - mean[0][gaussian_fit_index1]) ** 2) 

    mean[0][gaussian_fit_index1] = (1 - rho) * mean[0][gaussian_fit_index1] + rho * Gray_Frame[gaussian_fit_index1] 

    variance[0][gaussian_fit_index1] = (1 - rho) * variance[0][gaussian_fit_index1] + C  

# C is a constant 

    omega[0][gaussian_fit_index1] = (1 - alpha) * omega[0][gaussian_fit_index1] + alpha 

    omega[0][gaussian_not_fit_index1] = (1 - alpha) * omega[0][gaussian_not_fit_index1] 

 

    rho = alpha * norm_pdf(Gray_Frame[gaussian_fit_index2], mean[1][gaussian_fit_index2], sigma2[gaussian_fit_index2]) 

    C = rho * ((Gray_Frame[gaussian_fit_index2] - mean[1][gaussian_fit_index2]) ** 2) 

    mean[1][gaussian_fit_index2] = (1 - rho) * mean[1][gaussian_fit_index2] + rho * Gray_Frame[gaussian_fit_index2] 

    variance[1][gaussian_fit_index2] = (1 - rho) * variance[1][gaussian_fit_index2] + rho * C 

    omega[1][gaussian_fit_index2] = (1 - alpha) * omega[1][gaussian_fit_index2] + alpha 

    omega[1][gaussian_not_fit_index2] = (1 - alpha) * omega[1][gaussian_not_fit_index2] 

 

    rho = alpha * norm_pdf(Gray_Frame[gaussian_fit_index3], mean[2][gaussian_fit_index3], sigma3[gaussian_fit_index3]) 

    C = rho * ((Gray_Frame[gaussian_fit_index3] - mean[2][gaussian_fit_index3]) ** 2) 

    mean[2][gaussian_fit_index3] = (1 - rho) * mean[2][gaussian_fit_index3] + rho * Gray_Frame[gaussian_fit_index3] 

    variance[2][gaussian_fit_index3] = (1 - rho) * variance[2][gaussian_fit_index3] + C 

    omega[2][gaussian_fit_index3] = (1 - alpha) * omega[2][gaussian_fit_index3] + alpha 

    omega[2][gaussian_not_fit_index3] = (1 - alpha) * omega[2][gaussian_not_fit_index3] 

     

    # Updating the least probable gaussian for those pixel values which do not match any of the gaussian 

    mean[0][not_match_index] = Gray_Frame[not_match_index] 

    variance[0][not_match_index] = 200 

    omega[0][not_match_index] = 0.1 

 

    # normalise omega 

    sum = np.sum(omega,axis=0) 

    omega = omega/sum 

 

    # Finding omega by sigma values for ordering the gaussian 

    omega_by_sigma[0] = omega[0] / sigma1 

    omega_by_sigma[1] = omega[1] / sigma2 

    omega_by_sigma[2] = omega[2] / sigma3 

 

    # Getting order of indexes for sorting omega by sigma values 

    index = np.argsort(omega_by_sigma,axis=0) 

    back_index0 = np.where(omega_by_sigma[0]>T) 

    back_index1 = np.where(omega_by_sigma[1]>T) 

    back_index2 = np.where(omega_by_sigma[2]>T) 

     

    # From that index, sorting the mean, variance and omega 

    mean = np.take_along_axis(mean,index,axis=0) 

    variance = np.take_along_axis(variance,index,axis=0) 

    omega = np.take_along_axis(omega,index,axis=0) 

     

    Gray_Frame = Gray_Frame.astype(np.uint8) 

     

    # Getting background from the index1 and index2 

    bg[back_index1]=Gray_Frame[back_index1] 

    bg[back_index2]=Gray_Frame[back_index2] 

     

     

    cv.imshow('Gray_Frame',Gray_Frame) 

    cv.imshow('BackGround',bg) 

    cv.imshow('ForeGround',cv.absdiff(Gray_Frame,bg)) 

    out_1.write(bg) 

    out_2.write(cv.absdiff(Gray_Frame,bg)) 

     

capture.release() 

out_1.release() 

out_2.release() 

cv.destroyAllWindows() 